## A working lambda_function was created and connected with a flight_arrival_function in AWS Lambda Function services

In [ ]:
import json
import pandas as pd
import os
import sqlalchemy

def lambda_handler(event, context):
    
    import Flight_fn
    
    con = os.environ["con"]
    Airports = pd.read_sql_table("Airports", con=con)
    icao_list = pd.Series(Airports['icao']) 
    Flights = Flight_fn.flight_arrival_data(icao_list)
    Flights.to_sql('Flights',
              if_exists='append',
              con=con,
              index=False)
    
    return {
        'statusCode': 200,
        'body': json.dumps('Hello from Lambda!')
    }

In [ ]:
import pandas as pd
import requests
import json
import os


def cleaning_dataframe(df):
    df['Departure_Time']= pd.to_datetime(df['Departure_Time'], utc= True)
    df['Arrival_Time']= pd.to_datetime(df['Arrival_Time'], utc= True)
    df['Data_Retrieval_Timestamp']= pd.to_datetime(df['Data_Retrieval_Timestamp'])
    
    return df
    
    
def flight_arrival_data(icao_list):
    key = os.environ["Aerodata_key"]
    from datetime import datetime, date, timedelta
    from pytz import timezone
    Arrival_flights = []
    Flights_df = pd.DataFrame()
    for icao in icao_list:
        
        today = datetime.now().astimezone(timezone('Europe/Berlin')).date()
         
        tomorrow = (today + timedelta(days=1))
        
        times = [["00:00","11:59"],["12:00","23:59"]]
        
        for time in times:
            url = f"https://aerodatabox.p.rapidapi.com/flights/airports/icao/{icao}/{tomorrow}T{time[0]}/{tomorrow}T{time[1]}"
            querystring = {"withLeg":"true","direction":"Arrival","withCancelled":"false","withCodeshared":"true","withCargo":"false","withPrivate":"false"}
            
            headers = {
          'x-rapidapi-host': "aerodatabox.p.rapidapi.com",
          'x-rapidapi-key': key
          }
            
            response = requests.request("GET", url, headers=headers, params=querystring)
            print(response.status_code)
            arrival_flights_json = response.json()
            
            for hdj in arrival_flights_json['arrivals']:

                Arrival_data_dict ={}
                Arrival_data_dict['Arrival_ICAO']= icao
                Arrival_data_dict['Departure_Airport'] = hdj['departure']['airport'].get('name', None)
                Arrival_data_dict['Departure_Time'] = hdj.get('departure', {}).get('scheduledTime', {}).get('local', None)
                Arrival_data_dict['Arrival_Time'] = hdj.get('arrivals', {}).get('scheduledTime', {}).get('local', None)
                Arrival_data_dict['Flight_Status'] = hdj.get('status', None)
                Arrival_data_dict['Arrival_Terminal'] = hdj['arrival'].get('terminal', None)
                Arrival_data_dict['Flight_Number'] = hdj.get('number', None)
                Arrival_data_dict['Airline'] =hdj['airline'].get('name', None)
                Arrival_data_dict['Aircraft_type'] = hdj.get('aircraft',{}).get('model', None)
                
                    
                Arrival_data_dict['Data_Retrieval_Timestamp'] = datetime.now().astimezone(timezone('Europe/Berlin')).strftime("%d/%m/%Y %H:%M:%S")  
                
                Arrival_flights.append(Arrival_data_dict)
                
    return pd.DataFrame(Arrival_flights).pipe(cleaning_dataframe) 
    print("The Flight info has been updated")
